In [121]:
import pandas as pd
from imblearn.under_sampling import OneSidedSelection

In [122]:
train=pd.read_csv('./Dataset/Undersampling_0.33_train.csv',index_col=False, encoding='euc-kr')
test=pd.read_csv('./Dataset/test_시총_상장.csv',index_col=False,encoding='euc-kr')

In [123]:
X_train_int=train[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름',
'자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률']]

X_test_int=test[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름',
'자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률']]

In [124]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [125]:
 # One Side Selection을 사용하여 언더샘플링된 데이터 생성
oss = OneSidedSelection()
x_train_undersampled, y_train_undersampled = oss.fit_resample(X_test_int, y_test)

In [126]:
X_train_sc = X_train
X_test_sc = X_test

In [127]:
X_train_sum =X_train_sc
X_test_sum =X_test_sc

In [128]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import numpy as np

def perform_lgbm_grid_search(X_train, y_train, k_fold=5):
    # Stratified k-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # LGBM 하이퍼파라미터 후보 리스트 설정
    param_grid = {
        'max_depth': [2],
        'n_estimators': [100],
        'learning_rate': [0.01],
        'objective': ['binary'],
        'random_state': [0]
    }

    # LGBM 모델 초기화
    model = lgb.LGBMClassifier()

    # 그리드 서치 설정
    grid_search = GridSearchCV(model, param_grid, scoring='f1', cv=cv, verbose=1, n_jobs=-1)

    # 모델 학습 및 튜닝
    grid_search.fit(X_train, y_train)

    # 최적 하이퍼파라미터 출력
    print("Best Hyperparameters:", grid_search.best_params_)

    # 평균 평가 지표 계산
    mean_f1_score = np.mean(grid_search.cv_results_['mean_test_score'])
    print("Mean F1 Score:", mean_f1_score)

    return grid_search.best_params_, mean_f1_score

In [129]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_lgbm_with_best_params(X_train, y_train, X_test, y_test, best_params, k_fold=5):
    # Stratified k-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # LGBM 모델 초기화
    model = lgb.LGBMClassifier(**best_params)

    # 각 fold 별 평가 지표를 저장할 리스트 초기화
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # 모델 학습
        model.fit(X_train_fold, y_train_fold)

        # 테스트 데이터에 대한 예측 확률 얻기
        probabilities = model.predict_proba(X_test_fold)

        # threshold를 0.4로 설정하여 예측 클래스를 조정
        threshold = 0.4
        predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)

        # 평가 지표 계산
        accuracy = accuracy_score(y_test_fold, predicted_classes)
        precision = precision_score(y_test_fold, predicted_classes)
        recall = recall_score(y_test_fold, predicted_classes)
        f1 = f1_score(y_test_fold, predicted_classes)
        conf_matrix = confusion_matrix(y_test_fold, predicted_classes)

        # 각 fold 별 평가 지표를 리스트에 추가
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # 가장 좋은 f1-score 값을 가진 모델을 저장
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # 가장 좋은 f1-score 값을 가진 모델로 최종 예측 수행
    probabilities_final = best_model.predict_proba(X_test)
    y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)

    # 평가 지표 계산
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list

In [130]:
best_params, mean_f1_score = perform_lgbm_grid_search(X_train, y_train, k_fold=5)
evaluate_lgbm_with_best_params(X_train_int, y_train, x_train_undersampled, y_train_undersampled, best_params, k_fold=5)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\sklea

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 100, 'objective': 'binary', 'random_state': 0}
Mean F1 Score: 0.4863448275862069
Fold 1
Accuracy: 0.821917808219178
Precision: 0.7777777777777778
Recall: 0.3888888888888889
F1 score: 0.5185185185185185
Confusion Matrix:
[[53  2]
 [11  7]]
------------------------------
Fold 2
Accuracy: 0.7534246575342466
Precision: 0.5
Recall: 0.3888888888888889
F1 score: 0.43750000000000006
Confusion Matrix:
[[48  7]
 [11  7]]
------------------------------
Fold 3
Accuracy: 0.7638888888888888
Precision: 0.5555555555555556
Recall: 0.2777777777777778
F1 score: 0.3703703703703704
Confusion Matrix:
[[50  4]
 [13  5]]
------------------------------
Fold 4
Accuracy: 0.8333333333333334
Precision: 0.7142857142857143
Recall: 0.5555555555555556
F1 score: 0.6250000000000001
Confusion Matrix:
[[50  4]
 [ 8 10]]
------------------------------
Fold 5
Accuracy: 0.7777777777777778
Precision: 0.6
Recall: 0.3333333333333333
F1 score: 0.428571

C:\Users\dgh06\AppData\Local\Temp\ipykernel_27280\1646110371.py:66: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)


([0.821917808219178,
  0.7534246575342466,
  0.7638888888888888,
  0.8333333333333334,
  0.7777777777777778],
 [0.7777777777777778, 0.5, 0.5555555555555556, 0.7142857142857143, 0.6],
 [0.3888888888888889,
  0.3888888888888889,
  0.2777777777777778,
  0.5555555555555556,
  0.3333333333333333],
 [0.5185185185185185,
  0.43750000000000006,
  0.3703703703703704,
  0.6250000000000001,
  0.42857142857142855])

Final Test Results
Accuracy: 0.8055555555555556
Precision: 0.6875
Recall: 0.4074074074074074
F1 score: 0.5116279069767441
Confusion Matrix:
[[76  5]
 [16 11]]